# COVID-19 Dashboard
This dashboard visualises COVID-19 statistics, including daily cases and deaths. 

## Instructions
- Use the dropdown menu to select the metric to display.
- Click "Refresh Data" to fetch the latest statistics from the UKHSA API.

In [ ]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

plt.rcParams['figure.dpi'] = 100

API_BASE_URL = "https://api.ukhsa-dashboard.data.gov.uk"

# API requests
structure = {
    "theme": "infectious_disease",
    "sub_theme": "respiratory",
    "topic": "COVID-19",
    "geography_type": "Nation",
    "geography": "England"
}

metrics = {
    "cases": "COVID-19_cases_casesByDay",
    "admissions": "COVID-19_healthcare_admissionByDay",
    "deaths": "COVID-19_deaths_ONSByDay"
}

def construct_api_url(metric):
    return (
        f"{API_BASE_URL}/themes/{structure['theme']}/sub_themes/{structure['sub_theme']}"
        f"/topics/{structure['topic']}/geography_types/{structure['geography_type']}"
        f"/geographies/{structure['geography']}/metrics/{metric}"
    )

# Fetch data from the API
def fetch_data_from_api():
    all_dates = []
    data_dict = {"date": []}

    for name, code in metrics.items():
        print(f"Fetching {name} data...")
        try:
            url = construct_api_url(code)
            response = requests.get(url, timeout=10, params={"format": "json"})
            response.raise_for_status()
            results = response.json().get("results", [])
            
            dates = [item["date"] for item in results]
            values = [item["metric_value"] for item in results]

            if not all_dates:
                all_dates = dates
                data_dict["date"] = dates

            data_dict[name] = values

        except Exception as e:
            print(f"Error fetching {name}: {e}")
            data_dict[name] = [None] * len(all_dates)

    print("Data fetched successfully!")
    return pd.DataFrame(data_dict)

def load_local_data(file_path="weekly_data.json"):
    try:
        with open(file_path, "r") as file:
            data = json.load(file)
        return pd.DataFrame(data)
    except Exception as e:
        print(f"Error loading local data: {e}")
        return pd.DataFrame()  

def process_data(data):
    if data.empty:
        print("No valid data available. Using only local data.")
        return data
    data["date"] = pd.to_datetime(data["date"])
    return data.set_index("date")

global_data = process_data(load_local_data())

data_selector = wdg.Dropdown(
    options=global_data.columns.tolist() if not global_data.empty else [],
    value=global_data.columns[0] if not global_data.empty else None,
    description="Select Data:",
    disabled=global_data.empty,
)

plot_output = wdg.Output()

# Refresh the graph based on the selected data
def refresh_graph(change=None):
    with plot_output:
        clear_output(wait=True)
        if global_data.empty:
            print("No data available to plot.")
            return
        selected_column = data_selector.value
        if not selected_column:
            print("Please select a column to display.")
            return
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(global_data.index, global_data[selected_column], marker="o", label=selected_column)
        ax.grid(True, linestyle="--", alpha=0.6)
        ax.set_title(f"COVID-19 Weekly Data: {selected_column}", fontsize=14)
        ax.set_xlabel("Date", fontsize=12)
        ax.set_ylabel("Values", fontsize=12)
        ax.legend(loc="upper left")
        plt.show()

# Refresh data from the API
def refresh_button_callback(button):
    refresh_button.description = "Loading..."
    refresh_button.disabled = True
    global global_data

    current_selection = data_selector.value

    try:
        new_data = fetch_data_from_api()
        global_data = process_data(new_data)
        print(f"Data refresh completed.")
        data_selector.options = global_data.columns.tolist()

        if current_selection in data_selector.options:
            data_selector.value = current_selection
        else:
            data_selector.value = global_data.columns[0]

        refresh_graph()
    finally:
        refresh_button.description = "Refresh Data"
        refresh_button.disabled = False

refresh_button = wdg.Button(
    description="Refresh Data",
    disabled=False,
    button_style="primary",
    tooltip="Click to fetch current data from UKHSA API",
    icon="refresh",
)
refresh_button.on_click(refresh_button_callback)

data_selector.observe(refresh_graph, names="value")

dashboard_layout = wdg.VBox([refresh_button, data_selector, plot_output])
display(dashboard_layout)

refresh_graph()